## Jesuit communities in China ("Chrétientés") by Dehergne

Based on Dehergne, J. (1973). Répertoire des Jésuites de Chine de 1552 à 1800. Institutum historicum ; Letouzey & Ané. https://archive.org/details/bhsi37

Complemented with Dehergne, J. (1957). Les Chrétientés de Chine de la Période Ming (1581–1650). Monumenta Serica, 16(1–2), 1–136. https://doi.org/10.1080/02549948.1957.11730960 where more detailed information is provided.

The Répertoire includes two lists of Jesuits residences in China. One reports to 1644 and the second to 1701.

- [Planche: Carte des Chrétientés Chinoises de la fin des Ming (1644)](https://archive.org/details/bhsi37/page/398/mode/1up) and [map](https://archive.org/details/bhsi37/page/n390/mode/1up) Kleiotranscription at [sources/dehergne-locations-1644.cli](../sources/dehergne-locations-1644.cli)
- [VII. Carte des résidences de Chine en l'année 1701](https://archive.org/details/bhsi37/page/n400/mode/1up) and [map](https://archive.org/details/bhsi37/page/n392/mode/1up) kleio transcription at [sources/dehergne-locations-1701.cli](../sources/dehergne-locations-1701.cli).

### Overview
This notebook produces tables with the names of communties, their wkidata code, and comments related to
modern chinese spelling of the place name. 

The notebook also uses records related to place names in individual missionary entries in the "Repertoire"
to list all the people present in a community at a given point in time.

### Output

**Excel files produced in this notebook:**
- [residences-1644.xlsx](../inferences/residences-1644.xlsx)
- [residences-1701.xlsx](../inferences/residences-1701.xlsx)
- [residences-1644-1701.xlsx](../inferences/residences-1644-1701.xlsx)
- [residences-names-1644-1701.xlsx](../inferences/residences-names-1644-1701.xlsx)
- [residences-1644-1701-no-wikidata.xlsx](../inferences/residences-1644-1701-no-wikidata.xlsx)

**Map produced:**
- [residences_1644_map.html](../inferences/residences_1644_map.html)


### Setup

In [217]:
import timelink
import timelink.notebooks

print("Timelink version:", timelink.__version__)

tlnb = timelink.notebooks.TimelinkNotebook()
# tlnb.print_info()

Timelink version: 1.1.26


## List of residences in 1644

Loop through the list of residences in 1644 in file `dehergne-locations-1644.cli` and 
build a dataframe with the administrative structure and the wikidate of the locations. 


In [ ]:
import pandas as pd
from sqlalchemy import select
from dehergne_util import get_wikidata_id

geo1, geo2, geo3 = tlnb.db.get_model(['geo1','geo2','geo3'])
stmt = select(geo1).where(geo1.inside == 'deh-chre-1644').order_by(geo1.the_line)

place_list = []

with tlnb.db.session() as session:
    result = session.execute(stmt).fetchall()
    for province, in result:
        comment, wikidata = get_wikidata_id(province, if_missing='No wikidata')
        # print(province.id, province.name, wikidata, comment)
        place_list.append({'province': province.name,
                           'id': province.id,
                           'level': 'province',
                           'name': province.name,
                           'name_original': province.with_extra_info().name,
                           'province_wikidata_id': wikidata,
                           'wikidata_id': wikidata,
                           'the_line': province.the_line,
                           'comment': comment})
        fous = session.execute(select(geo2).where(geo2.inside == province.id)).fetchall()
        province_wikdata = wikidata
        for fou, in fous:
            the_type = fou.the_type
            comment, wikidata = get_wikidata_id(fou, if_missing='No wikidata')
            # print(' ', fou.name,  wikidata, comment)
            place_list.append({'province': province.name,
                               'province_id': province.id,
                               'province_wikidata_id': province_wikdata,
                               'id': fou.id,
                               'level': 'fou',
                               'fou':fou.name,
                               'name': fou.name,
                               'name_original': fou.with_extra_info().name,
                               'fou_wikidata_id': wikidata,
                               'wikidata_id': wikidata,
                               'the_line': fou.the_line,
                               'comment': comment})
            geo3s = session.execute(select(geo3).where(geo3.inside == fou.id)).fetchall()
            fou_wikidata = wikidata
            for tcheou_hien, in geo3s:
                comment, wikidata = get_wikidata_id(tcheou_hien, if_missing='No wikidata')
                # print('   ', tcheou_hien.name, wikidata,comment,  )
                place_list.append({'province': province.name,
                                    'province_id': province.id,
                                    'province_wikidata_id': province_wikdata,
                                    'fou_id': fou.id,
                                    'fou_wikidata_id': fou_wikidata,
                                    'id': tcheou_hien.id,
                                    'level': 'tcheou-hien',
                                    'fou':fou.name,
                                    'name': tcheou_hien.name,
                                    'name_original': tcheou_hien.with_extra_info().name,
                                    'wikidata_id': wikidata,
                                    'the_line': tcheou_hien.the_line,
                                    'comment': comment})


#### Create a dataframe with the locations in 1644

```{python}

In [ ]:
pd.set_option('display.max_rows', 300)
# create a dataframe from the list
places_1644_df = pd.DataFrame(place_list)
places_1644_df.info()
places_1644_df['year'] = 1644
cols=['year','id','level','province',  'fou','name', 'name_original', 'province_wikidata_id', 'fou_wikidata_id','wikidata_id', 'comment','the_line']
# 'the_line' is the order of the places in the list
places_1644_df[cols].sort_values(by=['the_line']).head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167 entries, 0 to 166
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   province              167 non-null    object
 1   id                    167 non-null    object
 2   level                 167 non-null    object
 3   name                  167 non-null    object
 4   name_original         167 non-null    object
 5   province_wikidata_id  167 non-null    object
 6   wikidata_id           167 non-null    object
 7   the_line              167 non-null    int64 
 8   comment               167 non-null    object
 9   province_id           154 non-null    object
 10  fou                   154 non-null    object
 11  fou_wikidata_id       154 non-null    object
 12  fou_id                105 non-null    object
dtypes: int64(1), object(12)
memory usage: 17.1+ KB


,year,id,level,province,fou,name,name_original,province_wikidata_id,fou_wikidata_id,wikidata_id,comment,the_line
0,1644,deh-r1644-chekiang,province,Chekiang,NaN,Chekiang,"Chekiang#Tche-kiang, today Zhejiang, 浙江, @wiki...",Q16967,NaN,Q16967,"Tche-kiang, today Zhejiang, 浙江, @dehergne:396",100
1,1644,deh-r1644-hangchou,fou,Chekiang,Hangchou,Hangchou,"Hangchou#Hang-tcheou, today Hangzhou, 杭州, @wik...",Q16967,Q4970,Q4970,"Hang-tcheou, today Hangzhou, 杭州,",101
2,1644,deh-r1644-fuyang,tcheou-hien,Chekiang,Hangchou,Fuyang,"Fuyang#Fou-yang, today Fuyang, 富阳, @wikidata:Q...",Q16967,Q4970,Q1011103,"Fou-yang, today Fuyang, 富阳,",105
3,1644,deh-r1644-jenho,tcheou-hien,Chekiang,Hangchou,Jenho,"Jenho#Jen-houo, today Renhe, 仁和县 (@wikidata:Q9...",Q16967,Q4970,Q9385136,"Jen-houo, today Renhe, 仁和县 (), Historical coun...",108
4,1644,deh-r1644-chuchow,fou,Chekiang,Chüchow,Chüchow,"Chüchow#""""""K'iu-tcheou, today Quzhou, 衢州, @wik...",Q16967,Q58235,Q58235,"K'iu-tcheou, today Quzhou, 衢州, , in the Chines...",111
5,1644,deh-r1644-huchow,fou,Chekiang,Huchow,Huchow,"Huchow#Hou-tcheou, today Huzhou, 湖州, @wikidata...",Q16967,Q42664,Q42664,"Hou-tcheou, today Huzhou, 湖州,",115
6,1644,deh-r1644-huchow-geo1-46,tcheou-hien,Chekiang,Huchow,Tehtsing,"Tehtsing%""""""""Tehtsing du Huchow, Té-ts'ingTech...",Q16967,Q42664,Q1191987,"""Tehtsing du Huchow, Té-ts'ingTeching# today D...",116
7,1644,deh-r1644-kashing,fou,Chekiang,Kashing,Kashing,"Kashing#Kia-hing, today Jiaxing, 嘉兴, @wikidata...",Q16967,Q58178,Q58178,"Kia-hing, today Jiaxing, 嘉兴,",120
8,1644,deh-r1644-kaosham,tcheou-hien,Chekiang,Kashing,Kashan,"Kashan#Kia-chan, today Jiashan, 嘉善, @wikidata:...",Q16967,Q58178,Q1361347,"Kia-chan, today Jiashan, 嘉善, Kaosham",122
9,1644,deh-r1644-tangsi,tcheou-hien,Chekiang,Kashing,Tangsi,"Tangsi#""""""""T'ang-k'i Tangchi"", today Tangqi, 塘...",Q16967,Q58178,Q10931032,"""T'ang-k'i Tangchi"", today Tangqi, 塘栖 , in the...",125


### Export the locations in 1644 

Export list of residences in 1644 to a file and list of wikidata
ids so they can be used to fetch coordinates and other information.

Ensure that notebook `wikidata-linked-data.ipynb` is run after the next cell to update cache of linked data.


In [220]:
places_1644_df.to_excel('../inferences/residences-1644.xlsx', index=False)
# save to wikidata ids directory
places_1644_df['wikidata_id'].to_csv('../inferences/wikidata-references/residences-1644.csv', index=False)

### Link with wikidata data

Requires that `wikidata-linked-data.ipynb` has been run first to generate the `locations_wikidata_info` file used here.


In [221]:
# load wikidata data
from dehergne_util import locations_wikidata_info_file

# load the locations wikidata info from xlsx file
print("Loading locations wikidata info from", locations_wikidata_info_file)
locations_wikidata_info = pd.read_excel(locations_wikidata_info_file, dtype=str)
locations_wikidata_info.set_index('wikidata_id', inplace=True)

# merge the two dataframes
merged_df = pd.merge(places_1644_df, locations_wikidata_info, on='wikidata_id', how='left')
cols = ['level', 'province', 'fou', 'name', 'wikidata_id', 'coordinates','latitude','longitude','comment', 'province_wikidata_id', 'fou_wikidata_id']
merged_df[cols].sort_values(by=['province', 'fou', 'name']).head(10)

Loading locations wikidata info from ../inferences/wikidata-references/locations_wikidata_info.xlsx


,level,province,fou,name,wikidata_id,coordinates,latitude,longitude,comment,province_wikidata_id,fou_wikidata_id
89,fou,Anhwei,Chuchow,Chuchow,Q114045,"(32.30621, 118.31148)",32.30621,118.31148,"Chuchow, today Chuzhou, 滁州,",Q40956,Q114045
90,fou,Anhwei,Hweichow,Hweichow,Q4358404,"(29.869722222222222, 118.42194444444445)",29.86972222222222,118.4219444444444,"Hoei-tcheou, today Huizhou, 徽州,",Q40956,Q4358404
92,tcheou-hien,Anhwei,Hweichow,Tungmen,No wikidata,NaN,NaN,NaN,"(?) In Dehergne(1957, p51): ""Chrétienté au Wuy...",Q40956,Q4358404
91,tcheou-hien,Anhwei,Hweichow,Wuyan hien,Q1357710,"(29.25, 117.85)",29.25,117.85,"Ou-yuen, today Wuyuan Xian, 婺源县, , Wuyuan hist...",Q40956,Q4358404
88,province,Anhwei,NaN,Anhwei,Q40956,"(31.833333333333, 117)",31.833333333333,117,"Anhui, Ngon-hoei, today Anhui, 安徽,",Q40956,NaN
4,fou,Chekiang,Chüchow,Chüchow,Q58235,"(28.95445, 118.8763)",28.95445,118.8763,"K'iu-tcheou, today Quzhou, 衢州, , in the Chines...",Q16967,Q58235
2,tcheou-hien,Chekiang,Hangchou,Fuyang,Q1011103,"(30.04998, 119.93697)",30.04998,119.93697,"Fou-yang, today Fuyang, 富阳,",Q16967,Q4970
1,fou,Chekiang,Hangchou,Hangchou,Q4970,"(30.25, 120.1675)",30.25,120.1675,"Hang-tcheou, today Hangzhou, 杭州,",Q16967,Q4970
3,tcheou-hien,Chekiang,Hangchou,Jenho,Q9385136,NaN,NaN,NaN,"Jen-houo, today Renhe, 仁和县 (), Historical coun...",Q16967,Q4970
5,fou,Chekiang,Huchow,Huchow,Q42664,"(30.8925, 120.0875)",30.8925,120.0875,"Hou-tcheou, today Huzhou, 湖州,",Q16967,Q42664


Missing coordinates

In [222]:
# list rows where coordinates are not available
missing_coords = merged_df[merged_df['coordinates'].isna()]
print(f"Rows with missing coordinates ({len(missing_coords)}):")
missing_coords[cols].sort_values(by=['province', 'fou', 'name'])


Rows with missing coordinates (27):


,level,province,fou,name,wikidata_id,coordinates,latitude,longitude,comment,province_wikidata_id,fou_wikidata_id
92,tcheou-hien,Anhwei,Hweichow,Tungmen,No wikidata,NaN,NaN,NaN,"(?) In Dehergne(1957, p51): ""Chrétienté au Wuy...",Q40956,Q4358404
3,tcheou-hien,Chekiang,Hangchou,Jenho,Q9385136,NaN,NaN,NaN,"Jen-houo, today Renhe, 仁和县 (), Historical coun...",Q16967,Q4970
16,tcheou-hien,Chekiang,Ningpo,Wuking,No wikidata,NaN,NaN,NaN,"(?) Uchim,Ou kin, Dehergne(1957) did not give ...",Q16967,Q42780
33,tcheou-hien,Fukien,Changchow,Aupua,Q14420305,NaN,NaN,NaN,"today Houban, 后坂 (), (@geonames:1977135), coor...",Q41705,Q68814
28,tcheou-hien,Fukien,Foochow,Niensien,No wikidata,NaN,NaN,NaN,"(?) Dehergne(1957, p30): ""A côté de Hai keu ...",Q41705,Q68481
43,tcheou-hien,Fukien,Funing,Lokia,No wikidata,NaN,NaN,NaN,"""""""Lukiahsiang,Lokahoeng, today Luojiaxiang, 罗...",Q41705,Q241877
45,tcheou-hien,Fukien,Funing,Tingteo,No wikidata,NaN,NaN,NaN,(?) 藤头？顶头？ in the Chinese translation it is re...,Q41705,Q241877
66,tcheou-hien,Fukien,Taiwan,Camarri,No wikidata,NaN,NaN,NaN,"today Jinbaoli, 金包里, in the Chinese translatio...",Q41705,Q22502
65,tcheou-hien,Fukien,Taiwan,Taparri,Q7420445,NaN,NaN,NaN,"""""""hoje：大包里 in the Chinese translation it is r...",Q41705,Q22502
81,tcheou-hien,Hukwang,Kingchow,Meng kia k´i,No wikidata,NaN,NaN,NaN,(?) in the Chinese translation it is recognize...,Q2235155,Q14135188


### Scan for coordinates

If there is no corresponding wikidata entry, or the wikidata entry does not have coordinates, but a geo location was obtained it is introduced in the comments of the transcription and can be retrived here.

In [223]:
import re
from dehergne_util import extract_coordinates

# set the coordinates for the places that have coordinates in the comment column
# interate rows with iterrows
for index, row in merged_df[merged_df['coordinates'].isna()].iterrows():
    # print()
    # print("Looking for coordinates of:", row['id'], row['name'])
    # if row['comment'] is not NaN and contains "coordinates", "latitude", or "longitude"
    if not re.search(r'coordinates:|latitude:|longitude:', row['comment'], flags=re.IGNORECASE):
        # print("No coordinates found in comment for row:", index, row['name'])
        continue
    coords = extract_coordinates(row['comment'])
    if coords:
        print()
        print(row['comment'])
        print("Found coordinates:", coords)
        merged_df.at[index, 'coordinates'] = coords
        merged_df.at[index, 'latitude'] = coords[0]
        merged_df.at[index, 'longitude'] = coords[1]



Jen-houo, today Renhe, 仁和县 (), Historical county name, coordinates: 30.448897N, 120.307504E
Found coordinates: (30.448897, 120.307504)

today Houban, 后坂 (), (@geonames:1977135), coordinates: 24.50213852506329N, 117.6917197408656EAu-poa,Heupuen
Found coordinates: (24.50213852506329, 117.6917197408656)

"""hoje：大包里 in the Chinese translation it is recognized as 塔巴里,in Dehergne(1957) it is noted as 大包里, which cannot be found in the map. Dehergne(1957) puts it as "Peninsule de Masu, à 1 lieue nord-ouest de San Salvador (社寮)"
                today's Santissima Trinidad ， with Masu coordinates:25.201218764354735N, 121.68618375811317E"""
Found coordinates: (25.201218764354735, 121.68618375811317)

today Jinbaoli, 金包里, in the Chinese translation it is recognized as “噶玛兰”, but in Dehergne(1957) it is noted as "金包里". It is an area in the Northern Taiwan, coordinates: 25.22189808588497N, 121.6382892794898EQuimauri
Found coordinates: (25.22189808588497, 121.6382892794898)

Huguang?, Hou-Quang, to

### List places with no coordinates

These are the ones that where not geo referenced so far.

In [224]:
# List rows in merged_df where the 'coordinates' column is NaN
missing_coords_df = merged_df[merged_df['coordinates'].isna()]
missing_coords_df

,province,id,level,name,name_original,province_wikidata_id,wikidata_id,the_line,comment,province_id,...,portuguese_description,coordinates,latitude,longitude,administrative_entity_id,administrative_entity_label_en,administrative_entity_label_zh,country_id,country_label,label
16,Chekiang,deh-r1644-wuking,tcheou-hien,Wuking,"Wuking#(?) Uchim,Ou kin, Dehergne(1957) did no...",Q16967,No wikidata,149,"(?) Uchim,Ou kin, Dehergne(1957) did not give ...",deh-r1644-chekiang,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,Fukien,deh-r1644-niensien,tcheou-hien,Niensien,"Niensien#""""""(?) Dehergne(1957, p30): ""A côté...",Q41705,No wikidata,184,"(?) Dehergne(1957, p30): ""A côté de Hai keu ...",deh-r1644-fukien,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43,Fukien,deh-r1644-lokia,tcheou-hien,Lokia,"Lokia#""""""Lukiahsiang,Lokahoeng, today Luojiaxi...",Q41705,No wikidata,232,"""""""Lukiahsiang,Lokahoeng, today Luojiaxiang, 罗...",deh-r1644-fukien,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,Fukien,deh-r1644-tingteo,tcheou-hien,Tingteo,"Tingteo#""""""(?) 藤头？顶头？ in the Chinese translati...",Q41705,No wikidata,240,(?) 藤头？顶头？ in the Chinese translation it is re...,deh-r1644-fukien,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81,Hukwang,deh-r1644-meng-kia-ki,tcheou-hien,Meng kia k´i,"Meng kia k´i#""""""(?) in the Chinese translation...",Q2235155,No wikidata,352,(?) in the Chinese translation it is recognize...,deh-r1644-hukwang,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,Anhwei,deh-r1644-tungment,tcheou-hien,Tungmen,"Tungmen#""""""(?) In Dehergne(1957, p51): ""Chréti...",Q40956,No wikidata,383,"(?) In Dehergne(1957, p51): ""Chrétienté au Wuy...",deh-r1644-anhwei,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111,Kiangsu,deh-r1644-kaokia,tcheou-hien,Kaokia,"Kaokia#""""""(?) today Gaojia, 高家, in the Chinese...",Q16963,No wikidata,444,"(?) today Gaojia, 高家, in the Chinese translati...",deh-r1644-kiangsu,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
151,Kwangtung,deh-r1644-hwanghsiaping,tcheou-hien,Hwanghsiaping,"Hwanghsiaping#""""""(?) Hwanghsiaping, today Vank...",Q15175,No wikidata,567,"(?) Hwanghsiaping, today Vankaxen, 黄下坪？, In th...",deh-r1644-kwangtung,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153,Kwangtung,deh-r1644-yangsiang,tcheou-hien,Yangsiang,"Yangsiang#""""""(?) In the Chinese translation, i...",Q15175,No wikidata,571,"(?) In the Chinese translation, it is recogniz...",deh-r1644-kwangtung,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Map the structure of residences in 1644

In [225]:
%pip install plotly

.bash_profile RUN!
Note: you may need to restart the kernel to use updated packages.


In [226]:
from textwrap import wrap
import plotly.graph_objects as go

# generate a dictionary for quick lookup with id as key
locations_wikidata_dict = {row['id']: row for index, row in merged_df.iterrows()}

# Define marker styles for each level
level_styles = {
    'province': {'color': 'red', 'size': 12},
    'fou': {'color': 'blue', 'size': 8},
    'tcheou-hien': {'color': 'green', 'size': 5}
}

fig = go.Figure()


for level, style in level_styles.items():
    fig.add_trace(go.Scattermap(
        lon=[None],
        lat=[None],
        mode='markers',
        marker=dict(color=style['color'], size=style['size']),
        name=level
    ))
# Add lines for 'fou' and 'tcheou_hien'
for _, row in merged_df.iterrows():
    level = row.get('level')
    if level == 'fou':
        id_origin = row.get('province_id', None)
        id_destination = row.get('id', None)
        lat_origin = locations_wikidata_dict.get(id_origin, {}).get('latitude', None)
        lon_origin = locations_wikidata_dict.get(id_origin, {}).get('longitude', None)
        lat_destination = locations_wikidata_dict.get(id_destination, {}).get('latitude', None)
        lon_destination = locations_wikidata_dict.get(id_destination, {}).get('longitude', None)
        if lat_origin and lon_origin and lat_destination and lon_destination:
            fig.add_trace(go.Scattermap(
                lon=[float(lon_origin), float(lon_destination)],
                lat=[float(lat_origin), float(lat_destination)],
                mode='lines',
                line=dict(width=1, color='red'),
                showlegend=False
            ))
    elif level == 'tcheou-hien':
        id_origin = row.get('fou_id', None)
        id_destination = row.get('id', None)
        lat_origin = locations_wikidata_dict.get(id_origin, {}).get('latitude', None)
        lon_origin = locations_wikidata_dict.get(id_origin, {}).get('longitude', None)
        lat_destination = locations_wikidata_dict.get(id_destination, {}).get('latitude', None)
        lon_destination = locations_wikidata_dict.get(id_destination, {}).get('longitude', None)
        if id_destination == 'Q7420445' or row['name'] == "Camarri": # special case for Taparri
            print(f"Debugging: {id_origin} -> {id_destination}")
            pass
        if lat_origin and lon_origin and lat_destination and lon_destination:
            fig.add_trace(go.Scattermap(
                lon=[float(lon_origin), float(lon_destination)],
                lat=[float(lat_origin), float(lat_destination)],
                mode='lines',
                line=dict(width=1, color='blue'),
                showlegend=False
            ))
# Add Markers for each location
for index, row in merged_df.iterrows():
    lat = row.get('latitude')
    lon = row.get('longitude')
    level = row.get('level')
    name = row.get('name')
    english_description = row.get('english_description', '')
    chinese_description = row.get('chinese_description', '')
    wikidata_id = row.get('wikidata_id', '')
    coordinates = row.get('coordinates', '')
    if name == "Camarri":
        print(f"Debugging: {name} with wikidata_id {wikidata_id}")

    if wikidata_id != 'No wikidata':
        english_label = row.get('english_label', '')
        chinese_label = row.get('chinese_label', '')
        portuguese_label = row.get('portuguese_label', '')
        english_description = row.get('english_description', '')
        chinese_description = row.get('chinese_description', '')
        portuguese_description = row.get('portuguese_description', '')
        wikidata_label = (
                         f"en:{english_label} ({english_description})<br>"
                         f"zh:{chinese_label} ({chinese_description})<br>"
                         f"pt:{portuguese_label} ({portuguese_description})"
        )
    else:
        wikidata_label = "No wikidata"
        english_label = ''
        chinese_label = ''
        portuguese_label = ''
        english_description = ''
        chinese_description = ''
        portuguese_description = ''

    name_original = row.get('name_original', '')
    comment = "<br>".join(wrap(name_original, width=40))
    hover_text = f"name: {name}<br>original: {comment}<br><br>wikidata ({wikidata_id})<br>{wikidata_label} <br>coordinates: {coordinates}"


    if level == 'tcheou-hien':
        pass
    if pd.notnull(lat) and pd.notnull(lon) and level in level_styles:
        style = level_styles[level]
        fig.add_trace(go.Scattermap(
            lon=[float(lon)],
            lat=[float(lat)],
            mode='markers+text',
            marker=dict(color=style['color'], size=style['size']),
            text=[name],
            textposition='top center',
            hovertext=[hover_text],
            hoverinfo='text',
            hoverlabel=dict(bgcolor='white', font_size=12),
            showlegend=False
        ))

# fig.update_layout(mapbox_style="open-street-map")
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_traces(textfont=dict(size=6))
fig.update_layout({
    "map": {"center": {"lat": 28.0,
                       "lon": 120.0},
            "zoom": 4,
            "style": "carto-positron",
            }
    },
    title= "Residences 1644",
    annotations=[dict(
        x=0,
        y=-0.1,
        xref="paper",
        yref="paper",
        text="<a href='https://archive.org/details/bhsi37/page/n390/mode/1up'>Source</a>",
        showarrow=False,
        font=dict(size=10)
    )],
    autosize=True,
    )

fig.show(config={"responsive": True})
fig.write_html("../inferences/residences_1644_map.html",
               config={"responsive": True},
               include_plotlyjs=True,
               auto_open=False)


Debugging: deh-r1644-taiwan -> deh-r1644-camarri
Debugging: Camarri with wikidata_id No wikidata


## List of residences in 1701

Loop through the list of residences in 1701 in file `dehergne-locations-1701.cli` and create a dataframe with the administrative structure and the wikidata of the locations.

In [227]:
import pandas as pd
from sqlalchemy import select
from dehergne_util import get_wikidata_id

geo1, geo2, geo3 = tlnb.db.get_model(['geo1','geo2','geo3'])
stmt = select(geo1).where(geo1.inside == 'deh-chre-1701').order_by(geo1.the_line)

place_list = []

with tlnb.db.session() as session:
    result = session.execute(stmt).fetchall()
    for province, in result:
        comment, wikidata = get_wikidata_id(province, if_missing='No wikidata')
        # print(province.id, province.name, wikidata, comment)
        place_list.append({'province': province.name,
                           'id': province.id,
                           'level': 'province',
                           'name': province.name,
                           'name_original': province.with_extra_info().name,
                           'province_wikidata_id': wikidata,
                           'wikidata_id': wikidata,
                           'the_line': province.the_line,
                           'comment': comment})
        fous = session.execute(select(geo2).where(geo2.inside == province.id)).fetchall()
        province_wikdata = wikidata
        for fou, in fous:

            comment, wikidata = get_wikidata_id(fou, if_missing='No wikidata')
            # print(' ', fou.name,  wikidata, comment)
            place_list.append({'province': province.name,
                               'province_id': province.id,
                               'province_wikidata_id': province_wikdata,
                               'id': fou.id,
                               'level': 'fou',
                               'fou':fou.name,
                               'name': fou.name,
                               'name_original': fou.with_extra_info().name,
                               'fou_wikidata_id': wikidata,
                               'wikidata_id': wikidata,
                               'the_line': fou.the_line,
                               'comment': comment})
            geo3s = session.execute(select(geo3).where(geo3.inside == fou.id)).fetchall()
            fou_wikidata = wikidata
            for tcheou_hien, in geo3s:
                comment, wikidata = get_wikidata_id(tcheou_hien, if_missing='No wikidata')
                # print('   ', tcheou_hien.name, wikidata,comment,  )
                place_list.append({'province': province.name,
                                    'province_id': province.id,
                                    'province_wikidata_id': province_wikdata,
                                    'fou_id': fou.id,
                                    'fou_wikidata_id': fou_wikidata,
                                    'id': tcheou_hien.id,
                                    'level': 'tcheou-hien',
                                    'fou':fou.name,
                                    'name': tcheou_hien.name,
                                    'name_original': tcheou_hien.with_extra_info().name,
                                    'wikidata_id': wikidata,
                                    'the_line': tcheou_hien.the_line,
                                    'comment': comment})


#### Create a dataframe with the locations in 1701



In [228]:
pd.set_option('display.max_rows', 300)
# create a dataframe from the list
places_1701_df = pd.DataFrame(place_list)
places_1701_df.info()
places_1701_df['year'] = 11701644
cols=['year','id','level','province',  'fou','name', 'name_original', 'province_wikidata_id', 'fou_wikidata_id','wikidata_id', 'comment','the_line']
# 'the_line' is the order of the places in the list
places_1701_df[cols].sort_values(by=['the_line']).head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   province              144 non-null    object
 1   id                    144 non-null    object
 2   level                 144 non-null    object
 3   name                  144 non-null    object
 4   name_original         144 non-null    object
 5   province_wikidata_id  144 non-null    object
 6   wikidata_id           144 non-null    object
 7   the_line              144 non-null    int64 
 8   comment               144 non-null    object
 9   province_id           129 non-null    object
 10  fou                   129 non-null    object
 11  fou_wikidata_id       129 non-null    object
 12  fou_id                56 non-null     object
dtypes: int64(1), object(12)
memory usage: 14.8+ KB


,year,id,level,province,fou,name,name_original,province_wikidata_id,fou_wikidata_id,wikidata_id,comment,the_line
0,11701644,deh-r1701-chekiang,province,Chekiang,NaN,Chekiang,Chekiang#Zhejiang 浙江， @wikidata: Q16967%CHEKIA...,Q16967,NaN,Q16967,Zhejiang 浙江， CHEKIANG (actuel ZHEJIANG),103
1,11701644,deh-r1701-hangchou,fou,Chekiang,Hangchou,Hangchou,"Hangchou#Hangzhou 杭州, @wikidata:Q4970",Q16967,Q4970,Q4970,"Hangzhou 杭州,",104
2,11701644,deh-r1701-hainang,tcheou-hien,Chekiang,Hangchou,Haining,Haining#海宁 @wikidata:Q286266,Q16967,Q4970,Q286266,海宁,110
3,11701644,deh-r1701-kashing,fou,Chekiang,Kashing,Kashing,"Kashing#Jiaxing 嘉兴, @wikidata:Q58178",Q16967,Q58178,Q58178,"Jiaxing 嘉兴,",113
4,11701644,deh-r1701-kinhwa,fou,Chekiang,Kinhwa,Kinhwa,"Kinhwa#Jinhua 金华, @wikidata:Q58210",Q16967,Q58210,Q58210,"Jinhua 金华,",116


### Export the locations in 1701 

Export list of residences in 1701 to a file and list of wikidata
ids so they can be used to fetch coordinates and other information.

Ensure that notebook `wikidata-linked-data.ipynb` is run after the next cell to update cache of linked data.


In [229]:
places_1701_df.to_excel('../inferences/residences-1701.xlsx', index=False)
# save to wikidata ids directory
places_1701_df['wikidata_id'].to_csv('../inferences/wikidata-references/residences-1701.csv', index=False)

#### Link with wikidata

Requires that `wikidata-linked-data.ipynb` has been run first to generate the `locations_wikidata_info` file used here.

In [230]:
# load wikidata data
from dehergne_util import locations_wikidata_info_file

# load the locations wikidata info from xlsx file (again, just in case it was updated
print("Loading locations wikidata info from", locations_wikidata_info_file)
locations_wikidata_info = pd.read_excel(locations_wikidata_info_file, dtype=str)
locations_wikidata_info.set_index('wikidata_id', inplace=True)

# merge the two dataframes
merged_df = pd.merge(places_1701_df, locations_wikidata_info, on='wikidata_id', how='left')
cols = ['level', 'province', 'fou', 'name', 'wikidata_id', 'coordinates','latitude','longitude','comment', 'province_wikidata_id', 'fou_wikidata_id']
merged_df[cols].sort_values(by=['province', 'fou', 'name']).head(10)

Loading locations wikidata info from ../inferences/wikidata-references/locations_wikidata_info.xlsx


,level,province,fou,name,wikidata_id,coordinates,latitude,longitude,comment,province_wikidata_id,fou_wikidata_id
2,tcheou-hien,Chekiang,Hangchou,Haining,Q286266,"(30.533333, 120.683333)",30.533333,120.683333,海宁,Q16967,Q4970
1,fou,Chekiang,Hangchou,Hangchou,Q4970,"(30.25, 120.1675)",30.25,120.1675,"Hangzhou 杭州,",Q16967,Q4970
3,fou,Chekiang,Kashing,Kashing,Q58178,"(30.7522, 120.75)",30.7522,120.75,"Jiaxing 嘉兴,",Q16967,Q58178
4,fou,Chekiang,Kinhwa,Kinhwa,Q58210,"(29.10462, 119.64937)",29.10462,119.64937,"Jinhua 金华,",Q16967,Q58210
5,tcheou-hien,Chekiang,Kinhwa,Lanchi,Q1023793,"(29.216667, 119.472222)",29.216667,119.472222,"Lanxi 兰溪,",Q16967,Q58210
6,fou,Chekiang,Ningpo,Ningpo,Q42780,"(29.875, 121.54916666667)",29.875,121.54916666667,"Ningbo 宁波,",Q16967,Q42780
7,tcheou-hien,Chekiang,Ningpo,Yenchow,Q1334217,"(29.81899, 121.5432)",29.81899,121.5432,"Yinzhou 鄞州,",Q16967,Q42780
0,province,Chekiang,NaN,Chekiang,Q16967,"(30.266666666667, 120.15)",30.266666666667,120.15,Zhejiang 浙江， CHEKIANG (actuel ZHEJIANG),Q16967,NaN
11,fou,Fukien,Changchow,Changchow,Q68814,"(24.50932, 117.66117)",24.50932,117.66117,"Zhangzhou 漳州,",Q41705,Q68814
13,tcheou-hien,Fukien,Chüanchow,Amoy,Q68744,"(24.479722222222, 118.08194444444)",24.479722222222,118.08194444444,"Xiamen 厦门,",Q41705,Q68695


#### Scan for coordinates

In [231]:
import re
from dehergne_util import extract_coordinates

# set the coordinates for the places that have coordinates in the comment column
# interate rows with iterrows
for index, row in merged_df[merged_df['coordinates'].isna()].iterrows():
    # print()
    # print("Looking for coordinates of:", row['id'], row['name'])
    # if row['comment'] is not NaN and contains "coordinates", "latitude", or "longitude"
    if not re.search(r'coordinates:|latitude:|longitude:', row['comment'], flags=re.IGNORECASE):
        # print("No coordinates found in comment for row:", index, row['name'])
        continue
    coords = extract_coordinates(row['comment'])
    if coords:
        print()
        print(row['comment'])
        print("Found coordinates:", coords)
        merged_df.at[index, 'coordinates'] = coords
        merged_df.at[index, 'latitude'] = coords[0]
        merged_df.at[index, 'longitude'] = coords[1]



Xinghua 兴化, , 兴化县，(no coord.), use the location the present-day Putian 莆田 Q68579, coordinates:25°26'21.98"N, 119°0'37.01"E
Found coordinates: (25.43943888888889, 119.01028055555555)

Kienning fou, Jianning 建寧,  (no coord.), using today's Jianyang wikidata:Q639862 coordinates:27°20'6"N, 118°6'58"E see 1644
Found coordinates: (27.334999999999997, 118.11611111111111)

湖广 , for coordinates we use Wuchang District wikidata:Q1014420, where the head quarters of vice-roy of Huguang (wikidata:Q11563236) were located, coordinates: 30°33'23"N, 114°18'38"E(actuel HUNAN + HUBEI)Huguang
Found coordinates: (30.55638888888889, 114.31055555555555)

KIANGNAN ou NANKIN (actuel ANHUI + JIANGSU), Jiangnan 江南, , coordinates:32°3'39"N, 118°46'44"E (Nankjing)
Found coordinates: (32.06083333333333, 118.77888888888889)

(confirm same as 1641 deh-r1644-shiuchow) @dehergne:n402 (p.359) Chao-tcheou, today Shaozhou, 韶州, , historical administrative division. The administrative center was Qujiang 曲江, wikidata:Q13466

### List places with no coordinates

These are the ones that where not geo referenced so far.

In [232]:
# List rows in merged_df where the 'coordinates' column is NaN
missing_coords_df = merged_df[merged_df['coordinates'].isna()]
missing_coords_df

,province,id,level,name,name_original,province_wikidata_id,wikidata_id,the_line,comment,province_id,...,portuguese_description,coordinates,latitude,longitude,administrative_entity_id,administrative_entity_label_en,administrative_entity_label_zh,country_id,country_label,label
36,Hukwang,deh-r1701-teian,fou,Teian,Teian#(?) No Wikidata,Q2235155,No wikidata,227,(?) No Wikidata,deh-r1701-hukwang,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75,Kiangsi,deh-r1701-tankiang,tcheou-hien,Tankiang,Tankiang#(?)No wikidata,Q57052,No wikidata,353,(?)No wikidata,deh-r1701-kiangsi,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119,Shansi,deh-r1701-pingyang-geo1-4,tcheou-hien,Wanchuan,"Wanchuan#""""""Wanquan 万泉县，the wikidata code of i...",Q46913,Q8475332,497,Wanquan 万泉县，the wikidata code of it does not c...,deh-r1701-shansi,...,NaN,NaN,NaN,NaN,Q10513631,Puzhou Fu,蒲州府,Q148,People's Republic of China,Wanquan / 萬泉縣
138,Shensi,deh-r1701-siao-tchai,tcheou-hien,Siao-tchai,"Siao-tchai#""""""(?)No wikidata, Dehergne 1957 «I...",Q47974,No wikidata,557,"(?)No wikidata, Dehergne 1957 «Il y a aussi un...",deh-r1701-shensi,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Map the structure of residences in 1701

In [233]:
from textwrap import wrap
import plotly.graph_objects as go

# generate a dictionary for quick lookup with id as key
locations_wikidata_dict = {row['id']: row for index, row in merged_df.iterrows()}

# Define marker styles for each level
level_styles = {
    'province': {'color': 'red', 'size': 12},
    'fou': {'color': 'blue', 'size': 8},
    'tcheou-hien': {'color': 'green', 'size': 5}
}

fig = go.Figure()


for level, style in level_styles.items():
    fig.add_trace(go.Scattermap(
        lon=[None],
        lat=[None],
        mode='markers',
        marker=dict(color=style['color'], size=style['size']),
        name=level
    ))
# Add lines for 'fou' and 'tcheou_hien'
for _, row in merged_df.iterrows():
    level = row.get('level')
    if level == 'fou':
        id_origin = row.get('province_id', None)
        id_destination = row.get('id', None)
        lat_origin = locations_wikidata_dict.get(id_origin, {}).get('latitude', None)
        lon_origin = locations_wikidata_dict.get(id_origin, {}).get('longitude', None)
        lat_destination = locations_wikidata_dict.get(id_destination, {}).get('latitude', None)
        lon_destination = locations_wikidata_dict.get(id_destination, {}).get('longitude', None)
        if lat_origin and lon_origin and lat_destination and lon_destination:
            fig.add_trace(go.Scattermap(
                lon=[float(lon_origin), float(lon_destination)],
                lat=[float(lat_origin), float(lat_destination)],
                mode='lines',
                line=dict(width=1, color='red'),
                showlegend=False
            ))
    elif level == 'tcheou-hien':
        id_origin = row.get('fou_id', None)
        id_destination = row.get('id', None)
        lat_origin = locations_wikidata_dict.get(id_origin, {}).get('latitude', None)
        lon_origin = locations_wikidata_dict.get(id_origin, {}).get('longitude', None)
        lat_destination = locations_wikidata_dict.get(id_destination, {}).get('latitude', None)
        lon_destination = locations_wikidata_dict.get(id_destination, {}).get('longitude', None)

        if lat_origin and lon_origin and lat_destination and lon_destination:
            fig.add_trace(go.Scattermap(
                lon=[float(lon_origin), float(lon_destination)],
                lat=[float(lat_origin), float(lat_destination)],
                mode='lines',
                line=dict(width=1, color='blue'),
                showlegend=False
            ))
# Add Markers for each location
for index, row in merged_df.iterrows():
    lat = row.get('latitude')
    lon = row.get('longitude')
    level = row.get('level')
    name = row.get('name')
    english_description = row.get('english_description', '')
    chinese_description = row.get('chinese_description', '')
    wikidata_id = row.get('wikidata_id', '')
    coordinates = row.get('coordinates', '')
    if name == "Camarri":
        print(f"Debugging: {name} with wikidata_id {wikidata_id}")

    if wikidata_id != 'No wikidata':
        english_label = row.get('english_label', '')
        chinese_label = row.get('chinese_label', '')
        portuguese_label = row.get('portuguese_label', '')
        english_description = row.get('english_description', '')
        chinese_description = row.get('chinese_description', '')
        portuguese_description = row.get('portuguese_description', '')
        wikidata_label = (
                         f"en:{english_label} ({english_description})<br>"
                         f"zh:{chinese_label} ({chinese_description})<br>"
                         f"pt:{portuguese_label} ({portuguese_description})"
        )
    else:
        wikidata_label = "No wikidata"
        english_label = ''
        chinese_label = ''
        portuguese_label = ''
        english_description = ''
        chinese_description = ''
        portuguese_description = ''

    name_original = row.get('name_original', '')
    comment = "<br>".join(wrap(name_original, width=40))
    hover_text = f"name: {name}<br>original: {comment}<br><br>wikidata ({wikidata_id})<br>{wikidata_label} <br>coordinates: {coordinates}"


    if level == 'tcheou-hien':
        pass
    if pd.notnull(lat) and pd.notnull(lon) and level in level_styles:
        style = level_styles[level]
        fig.add_trace(go.Scattermap(
            lon=[float(lon)],
            lat=[float(lat)],
            mode='markers+text',
            marker=dict(color=style['color'], size=style['size']),
            text=[name],
            textposition='top center',
            hovertext=[hover_text],
            hoverinfo='text',
            hoverlabel=dict(bgcolor='white', font_size=12),
            showlegend=False
        ))

# fig.update_layout(mapbox_style="open-street-map")
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_traces(textfont=dict(size=6))
fig.update_layout({
    "map": {"center": {"lat": 28.0,
                       "lon": 120.0},
            "zoom": 4,
            "style": "open-street-map",
            }
    },
    title= "Residences 1701",
    annotations=[dict(
        x=0,
        y=-0.1,
        xref="paper",
        yref="paper",
        text="<a href='https://archive.org/details/bhsi37/page/n392/mode/1up'>Source</a>",
        showarrow=False,
        font=dict(size=10)
    )],
    autosize=True,
    )

fig.show(config={"responsive": True})
fig.write_html("../inferences/residences_1701_map.html",
               config={"responsive": True},
               include_plotlyjs=True,
               auto_open=False)


## Append the two lists 1644 and 1701


In [234]:
# append places_1644_df and places_1701_df into a single dataframe
places_1644_1701_df = pd.concat([places_1644_df, places_1701_df], ignore_index=True)
# export to excel
places_1644_1701_df.to_excel('../inferences/residences-1644-1701.xlsx', index=False)
# export to csv wikidata ids directory
places_1644_1701_df['wikidata_id'].to_csv('../inferences/wikidata-references/residences-1644-1701.csv', index=False)

In [235]:
places_1644_1701_df.head()

,province,id,level,name,name_original,province_wikidata_id,wikidata_id,the_line,comment,province_id,fou,fou_wikidata_id,fou_id,year
0,Chekiang,deh-r1644-chekiang,province,Chekiang,"Chekiang#Tche-kiang, today Zhejiang, 浙江, @wiki...",Q16967,Q16967,100,"Tche-kiang, today Zhejiang, 浙江, @dehergne:396",NaN,NaN,NaN,NaN,1644
1,Chekiang,deh-r1644-hangchou,fou,Hangchou,"Hangchou#Hang-tcheou, today Hangzhou, 杭州, @wik...",Q16967,Q4970,101,"Hang-tcheou, today Hangzhou, 杭州,",deh-r1644-chekiang,Hangchou,Q4970,NaN,1644
2,Chekiang,deh-r1644-fuyang,tcheou-hien,Fuyang,"Fuyang#Fou-yang, today Fuyang, 富阳, @wikidata:Q...",Q16967,Q1011103,105,"Fou-yang, today Fuyang, 富阳,",deh-r1644-chekiang,Hangchou,Q4970,deh-r1644-hangchou,1644
3,Chekiang,deh-r1644-jenho,tcheou-hien,Jenho,"Jenho#Jen-houo, today Renhe, 仁和县 (@wikidata:Q9...",Q16967,Q9385136,108,"Jen-houo, today Renhe, 仁和县 (), Historical coun...",deh-r1644-chekiang,Hangchou,Q4970,deh-r1644-hangchou,1644
4,Chekiang,deh-r1644-chuchow,fou,Chüchow,"Chüchow#""""""K'iu-tcheou, today Quzhou, 衢州, @wik...",Q16967,Q58235,111,"K'iu-tcheou, today Quzhou, 衢州, , in the Chines...",deh-r1644-chekiang,Chüchow,Q58235,NaN,1644


## Show one location

In [236]:
entity_id = 'deh-r1644-chuchow'
with tlnb.db.session() as session:
    # save the dataframe to the database
    loc = tlnb.db.get_entity(entity_id, session)
    if loc is not None:
        print(loc.to_kleio())

geo2$Chüchow#"""K'iu-tcheou, today Quzhou, 衢州, @wikidata:Q58235, in the Chinese translation it is recognized as “遂州”, which is wrong, both phonetically and geographically. In Dehergne(1957), it is noted as "衢州"."""/geo2/obs="""      in the Chinese translation it is recognized as “遂州”, which is wrong, both
  phonetically and geographically. In Dehergne(1957), it is noted as "衢州".
  """
  atr$activa/sim/1613
  atr$geoentity:name@wikidata/"https://www.wikidata.org/wiki/Q58235"#"""K'iu-tcheou, today Quzhou, 衢州, @wikidata:Q58235, in the Chinese translation it is recognized as “遂州”, which is wrong, both phonetically and geographically. In Dehergne(1957), it is noted as "衢州"."""%Q58235/1644


## Listing people in residences

In [237]:
import re
import pandas as pd
from sqlalchemy import select

from timelink.kleio.utilities import convert_timelink_date, format_timelink_date
from timelink.api.models import Geoentity

nattributes = tlnb.db.get_view('nattributes')

def show_nattributes_for_wikidata(wikidata, ident='', session=None):
    """ Show the nattribute for a given wikidata id
    """
    wikidata_uri = 'https://www.wikidata.org/wiki/' + wikidata
    stmt = select(nattributes).where(nattributes.c.the_value == wikidata_uri).order_by(nattributes.c.the_date)
    results = session.execute(stmt).fetchall()
    for result in results:
        fdate = format_timelink_date(result.the_date)
        print(f"{ident}{fdate} {result.name} ({result.the_type}) {result.id} {result.pobs} ({result.the_date})")


In [238]:

geo1, geo2, geo3 = tlnb.db.get_model(['geo1', 'geo2', 'geo3'])
stmt = select(geo1).order_by(geo1.the_source, geo1.the_line).where(geo1.inside.in_(['deh-chre-1644','deh-chre-1701']))

place_list = []
name_list = []
with tlnb.db.session() as session:
    result = session.execute(stmt).fetchall()
    for province, in result:
        print()
        comment, wikidata = get_wikidata_id(province, if_missing='No wikidata')
        print(province.name, wikidata, comment)
        place_list.append({'id': province.id, 'level': 'province', 'province': province.name,'fou':'',  'name': province.name, 'wikidata': wikidata, 'comment': comment})
        show_nattributes_for_wikidata(wikidata, '  ', session=session)
        # search all the entities that have this value in one of the attributes
        fous = session.execute(select(geo2).where(geo2.inside == province.id)).fetchall()
        for fou, in fous:
            print()
            comment, wikidata = get_wikidata_id(fou, if_missing='No wikidata')
            print(' ', fou.name,  wikidata, comment)
            place_list.append({'id': fou.id, 'level': 'fou', 'province': province.name, 'fou':fou.name,  'name': fou.name, 'wikidata': wikidata, 'comment': comment})
            show_nattributes_for_wikidata(wikidata, '   ', session=session)
            geo3s = session.execute(select(geo3).where(geo3.inside == fou.id)).fetchall()
            for tcheou_hien, in geo3s:
                print()
                comment, wikidata = get_wikidata_id(tcheou_hien, if_missing='No wikidata')
                print('   ', tcheou_hien.name, wikidata,comment,  )
                place_list.append({'id': tcheou_hien.id, 'level': 'tcheou-hien', 'province': province.name, 'fou':fou.name,  'name': tcheou_hien.name, 'wikidata': wikidata, 'comment': comment})
                show_nattributes_for_wikidata(wikidata, '     ', session=session)


Chekiang Q16967 Tche-kiang, today Zhejiang, 浙江,  @dehergne:396
  1617-10-02 Gaspar Ferreira (estadia@wikidata) deh-gaspar-ferreira  (16171002)
  >1621 Álvaro Semedo (estadia@wikidata) deh-alvaro-semedo  (16210000.3)
  1628 Pedro Ribeiro (estadia@wikidata) deh-pedro-ribeiro  (16280000)
  <1663 Philippe Couplet (estadia@wikidata) deh-philippe-couplet  (16629999.7)
  1688 Paul Banhes Wan (estadia@wikidata) deh-paul-banhes-wan  (16880000)
  >1695 Antonio Faglia (estadia@wikidata) deh-antonio-faglia  (16950000.3)
  >1705 Antonio Faglia (estadia@wikidata) deh-antonio-faglia  (17050000.3)
  1707 Cyr Contancin (estadia@wikidata) deh-cyr-contancin  (17070000)
  1710 Joseph Marie Anne de Moyriac Mailla (estadia@wikidata) deh-joseph-marie-anne-de-moyriac-mailla  (17100000)
  >1710 Jan Baptista Chrzciciel Bakowski (estadia@wikidata) deh-jan-baptista-chrzciciel-bakowski  (17100000.3)
  1713 Romain Hinderer (estadia@wikidata) deh-romain-hinderer  (17130000)
  1714 Romain Hinderer (estadia@wikidata)

### Create a dataframe with the people in residences

Loop through the list of residences in 1644 and 1701 and create a dataframe with the people in each residence.